# Requirements 📋✅

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
# import cv2

# Needed for the data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Needed for the CNN
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import image_dataset_from_directory


In [3]:
from tensorflow.keras.callbacks import EarlyStopping


In [4]:
from tensorflow import convert_to_tensor


# Hopefully fixing cuda

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [6]:
!echo $CUDA_VISIBLE_DEVICES


-1


# The Dataset location 📍

## Label Data Load ⏳

In [7]:
# params
batch_size = 32
img_height = 256
img_width = 256
validation_split = 0.2
num_classes = 4


In [8]:
train_dataset = image_dataset_from_directory(
    directory='/home/kyrill/code/pt-ai/pt-ai/raw_data/processed_data_03',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    validation_split=validation_split,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


Found 4772 files belonging to 4 classes.
Using 3818 files for training.


2023-11-30 21:06:46.079472: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [9]:
validation_dataset = image_dataset_from_directory(
    directory='/home/kyrill/code/pt-ai/pt-ai/raw_data/processed_data_03',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    validation_split=validation_split,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


Found 4772 files belonging to 4 classes.
Using 954 files for validation.


In [10]:
type(train_dataset)


tensorflow.python.data.ops.prefetch_op._PrefetchDataset

## Preprocessing ⚙️

### CNN Model Function 🦾💻🧠

In [11]:
def initialize_CNN():
    model = models.Sequential()

    # Preprocessing layers
    model.add(layers.CenterCrop(height=350, width=450, input_shape=[256,256,1]))

    # Build of the Model
    model.add(layers.Conv2D(filters=8, kernel_size=(4,4), activation='relu', padding='same'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Conv2D(16,(3,3), activation='relu'))
    model.add(layers.MaxPool2D((2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(4, activation='softmax'))

    # Compilation of the Model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [12]:
# Summary of the CNN model
tmp_first_CNN = initialize_CNN()
tmp_first_CNN.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 center_crop (CenterCrop)    (None, 350, 450, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 350, 450, 8)       136       
                                                                 
 max_pooling2d (MaxPooling2  (None, 175, 225, 8)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 173, 223, 16)      1168      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 86, 111, 16)       0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 152736)            0

In [13]:
es = EarlyStopping(patience=30, restore_best_weights=True)


In [15]:
history = tmp_first_CNN.fit(train_dataset,
                            epochs=100,
                            validation_data=validation_dataset,
                            batch_size=16,
                            callbacks=[es])


Epoch 1/100
120/120 [==============================] - 102s 845ms/step - loss: 1.3767 - accuracy: 0.3442 - val_loss: 1.3642 - val_accuracy: 0.3522
Epoch 2/100
 35/120 [=======>......................] - ETA: 1:16 - loss: 1.3646 - accuracy: 0.3509

KeyboardInterrupt: 